In [1]:
import numpy as np
f = open('day_11_input.txt')
input_list = f.read().split('\n')
input_array = np.array([list(item) for item in input_list])

In [2]:
input_array

array([['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', '.', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ...,
       ['.', 'L', 'L', ..., '.', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L'],
       ['L', 'L', 'L', ..., 'L', 'L', 'L']], dtype='<U1')

In [3]:
def find_neighbors(x_coordinate, y_coordinate, inp_array):
    
    valid_x_neighbors = [x_coordinate + x for x in range(-1, 2) if (x_coordinate + x) >= 0 and (x_coordinate + x) < len(inp_array[0])]
    valid_y_neighbors = [y_coordinate + y for y in range(-1, 2) if (y_coordinate + y) >= 0 and (y_coordinate + y) < len(inp_array)]
    
    neighbor_coordinates = [[x, y] for x in valid_x_neighbors for y in valid_y_neighbors if not ((x == x_coordinate) and (y == y_coordinate))]
    neighbor_list = []

    for x_neighbor_coordinate, y_neighbor_coordinate in neighbor_coordinates:
        neighbor_list.append(inp_array[y_neighbor_coordinate, x_neighbor_coordinate])
    
    return np.array(neighbor_list)

In [4]:
def generation(inp_array):
    array_copy = inp_array.copy()
    counter = 0
    for y_index,row in enumerate(inp_array):
        for x_index,item in enumerate(row):
            neighbor_array = find_neighbors(x_index, y_index, inp_array)
            if item == "L" and ("#" not in neighbor_array):
                array_copy[y_index, x_index] = "#"
                counter += 1
            elif item == "#" and len(neighbor_array[neighbor_array == "#"]) >= 4:
                array_copy[y_index, x_index] = "L"
                counter += 1
    
    return array_copy, counter

In [5]:
def day_11_part_1():
    array_copy = input_array.copy()
    counter = 1
    while counter > 0:
        array_copy, counter = generation(array_copy)
    
    return len(array_copy[array_copy == '#'])

In [6]:
day_11_part_1()

2238

In [6]:
full_array = []
for y, row in enumerate(input_array):
    new_row = []
    for x, item in enumerate(row):
        neighbor_coordinate_list = []
        #west
        counter = 1
        while (x - counter) >= 0:
            candidate = input_array[y, x - counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x - counter, y])
                break
        
        #east
        counter = 1
        while (x + counter) < len(input_array[0]):
            candidate = input_array[y, x + counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x + counter, y])
                break
        
        #north
        counter = 1
        while (y - counter) >= 0:
            candidate = input_array[y - counter, x]
            if candidate != '.':
                neighbor_coordinate_list.append([x, y - counter])
                break
        
        #south
        counter = 1
        while (y + counter) < len(input_array):
            candidate = input_array[y + counter, x]
            if candidate != '.':
                neighbor_coordinate_list.append([x, y + counter])
                break

        #northwest
        counter = 1
        while ((x - counter) >= 0) and ((y - counter) >= 0):
            candidate = input_array[y - counter, x - counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x - counter, y - counter])
                break
        
        #northeast
        counter = 1
        while ((x + counter) < len(input_array[0])) and ((y - counter) >= 0):
            candidate = input_array[y - counter, x + counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x + counter, y - counter])
                break
        
        #southwest
        counter = 1
        while ((x - counter) >= 0) and ((y + counter) < len(input_list)):
            candidate = input_array[y + counter, x - counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x - counter, y + counter])
                break
        
        #southeast
        counter = 1
        while ((x - counter) < len(input_array[0])) and ((y + counter) < len(input_list)):
            candidate = input_array[y + counter, x + counter]
            if candidate != '.':
                neighbor_coordinate_list.append([x + counter, y + counter])
                break

        new_row.append(np.array(neighbor_coordinate_list))
    full_array.append(np.array(new_row))

KeyboardInterrupt: 

In [8]:
new_row

[array([[1, 0],
        [0, 1],
        [1, 1]]),
 array([[0, 0],
        [2, 0],
        [1, 1],
        [0, 1],
        [2, 1]]),
 array([[1, 0],
        [3, 0],
        [2, 1],
        [1, 1],
        [3, 1]]),
 array([[2, 0],
        [4, 0],
        [3, 1],
        [2, 1],
        [4, 1]]),
 array([[3, 0],
        [5, 0],
        [4, 1],
        [3, 1],
        [5, 1]]),
 array([[4, 0],
        [6, 0],
        [5, 1],
        [4, 1],
        [6, 1]]),
 array([[5, 0],
        [7, 0],
        [6, 1],
        [5, 1],
        [7, 1]]),
 array([[6, 0],
        [8, 0],
        [7, 1],
        [6, 1],
        [8, 1]]),
 array([[7, 0],
        [9, 0],
        [8, 1],
        [7, 1],
        [9, 1]])]